# Proyecto 2 Ironhack. 
Analisis de Lista de Reproduccion. Generacion de una lista de reproduccion nueva

In [53]:
import base64
import requests
import datetime
from urllib.parse import urlencode
import pandas as pd

## 1. Coneccion con la API de Spotify

In [54]:
# Llaves de acceso para conectarnos a la API de Spotify
client_id = '8e5f91c2fc27453eb255a039b42bcb70'
client_secret = '09645aff5c8849f2baab977e4cfd1017'
user_id = '1279741536'

In [55]:
# Generamos esta classe que nos funcionará para que de manera automatica se genere toda la solicitud de acceso a la API

class SpotifyAPI(object):
    access_token = None
    access_token_expires = datetime.datetime.now()
    client_id = None
    client_secret = None
    access_token_did_expire = True
    token_url = 'https://accounts.spotify.com/api/token'
    
    def __init__(self, client_id, client_secret, *args, **kwargs):
        super().__init__(*args,**kwargs)
        self.client_id = client_id
        self.client_secret = client_secret
    
    # Aqui se crean las credenciales en base 64 para poder acceder a la API
    def get_client_credentials(self):
        """ returns a base64 encoded string """
        client_id = self.client_id
        client_secret = self.client_secret
        if client_secret == None or client_id == None:
            raise Exception("You must set client_id and client_secret")
            
        client_creds = f"{client_id}:{client_secret}"
        client_creds_b64 = base64.b64encode(client_creds.encode())        
        return client_creds_b64.decode()
    
    # Aqui estamos generando los headers de acceso
    def get_token_headers(self):
        client_creds_b64 = self.get_client_credentials()
        
        return {
            "Authorization":f"Basic {client_creds_b64}"
        }
    
    def get_token_data(self):
        return {
            "grant_type":"client_credentials"
        }
    
    # Aqui nos autentificamos y obtenemos el token de acceso
    def perform_auth(self):
        token_url = self.token_url
        token_data = self.get_token_data()
        token_headers = self.get_token_headers()
        
        r = requests.post(token_url, data = token_data, headers = token_headers)

        if r.status_code not in range(200,299):
            raise Exceptiontion('Could not authenticate client')
        
            
        data = r.json()
        now = datetime.datetime.now()
        acces_token = data['access_token']
        expires_in = data['expires_in'] #seconds
        expires = now + datetime.timedelta(seconds = expires_in)
        self.access_token = acces_token
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now
        return True
    
    # Extraemos el token de acceso
    def get_access_token(self):
        token = self.access_token
        expires = self.access_token_expires
        now = datetime.datetime.now()
        if expires < now:
            self.perform_auth()
            return self.get_access_token()
        elif token == None:
            self.perform_auth()
            return self.get_access_token()
        return token
    
    # Aqui son los tokens de para las busquedas
    def get_resource_header(self):
        access_token = self.get_access_token()
        headers = {
            "Authorization":f"Bearer {access_token}"
        }
        return headers
        
    # busqueda de listas de reproduccion
    def get_user_playlist(self,u_id):
        endpoint = f'https://api.spotify.com/v1/users/{user_id}/playlists'
        headers = self.get_resource_header()
        r = requests.get(endpoint,headers=headers)
        if r.status_code not in range(200,299):
            return {}
        return r.json()
    
    # Busqueda de las canciones por playlist
    def get_song_playlist(self,playlist_id):
        endpoint = f'https://api.spotify.com/v1/playlists/{playlist_id}/tracks'
        headers = self.get_resource_header()
        r = requests.get(endpoint,headers=headers)
        if r.status_code not in range(200,299):
            return {}
        return r.json()
    
    # Busqueda de las canciones por playlist
    def get_audio_features(self,song_id):
        endpoint = f'https://api.spotify.com/v1/audio-features/{song_id}'
        headers = self.get_resource_header()
        r = requests.get(endpoint,headers=headers)
        if r.status_code not in range(200,299):
            return {}
        return r.json()

In [56]:
# Llamamos a la funcion y le asignamos nuestros client ID y Client Secret
spotify = SpotifyAPI(client_id,client_secret)

In [57]:
# Nos indica si fue realizada la coneccion con la API
spotify.perform_auth()

True

In [58]:
# Obtenemos el access_token que nos servirá para las consultas nescesarias. Por eso lo guardamos en una variable y que sea mas
# sencillo llamarlo posteriormente.
access_token = spotify.access_token
access_token

'BQC-P2hmNedGn0qC3e9KdL_my0ibM2KrqhBnkoqqfXVk4PVYRJTKff-zaD8rBHxNNlBNIQexpYxwLb2qFJ0'

## 2. Busqueda y descarga de la informacion de las listas de reproduccion

In [59]:
# Obtenemos las listas de reproduccion en mi usuario
playlist_rikrdinii = spotify.get_user_playlist(user_id)
columns = ['name','id','tracks']
pl_rikrdinii = pd.DataFrame(playlist_rikrdinii['items'],columns=columns)
pl_rikrdinii.head()


,name,id,tracks
0,Mis pistas en Shazam,4cneXKJlVKFLKAYuwtk57I,{'href': 'https://api.spotify.com/v1/playlists...
1,Pure Rock & Roll,37i9dQZF1DWWRktbhJiuqL,{'href': 'https://api.spotify.com/v1/playlists...
2,Fiesta Indie,37i9dQZF1DWSXAHZTiX76o,{'href': 'https://api.spotify.com/v1/playlists...
3,Radio Rock,37i9dQZF1DX1te6miphixI,{'href': 'https://api.spotify.com/v1/playlists...
4,Songs For Coding (Canciones para Programar),1Yw7Hve6LoD46LlvfGI29T,{'href': 'https://api.spotify.com/v1/playlists...


In [60]:
# Generamos una lista con los ID de cada Lista de reproduccion
id_playlist = [playlist_rikrdinii['items'][i]['id'] for i in range(len(playlist_rikrdinii['items']))]
id_playlist[4]

'1Yw7Hve6LoD46LlvfGI29T'

In [80]:
# Analizamos una lista de reproduccion mediante su ID
#lista_1 = spotify.get_song_playlist('7M3zmWRA291rxcI1vw47DT')

# Obtenemos la info de las canciones en la lista de reproduccion
canciones = []
for a in id_playlist:
    lista_1 = spotify.get_song_playlist(a)
    for i in range(len(lista_1['items'])):
        id_cancion = lista_1['items'][i]['track']['id']
        nombre_cancion = lista_1['items'][i]['track']['name']
        album =lista_1['items'][i]['track']['album']['name']
        artista = lista_1['items'][i]['track']['artists'][0]['name']

        canciones.append([id_cancion,nombre_cancion,album,artista])

df = pd.DataFrame(canciones,columns =['id','nombre','album','artista'])
df

# Hacer eliminacion de ID que sean igual a None o Nulos

,id,nombre,album,artista
0,7gHs73wELdeycvS48JfIos,Faded,Faded,Alan Walker
1,5khPVYWIdAE0kSTqekYIwB,Blah Blah Blah,Blah Blah Blah EP,Armin van Buuren
2,3JOVTQ5h8HGFnDdp4VT3MP,Mad World (Feat. Michael Andrews),Trading Snakeoil for Wolftickets,Gary Jules
3,3JOVTQ5h8HGFnDdp4VT3MP,Mad World (Feat. Michael Andrews),Trading Snakeoil for Wolftickets,Gary Jules
4,1Ob57NqUydp5cC3FCNV4Gy,I Feel You - 2006 Remaster,Songs of Faith and Devotion (2006 Remaster),Depeche Mode
...,...,...,...,...
538,4UWq0SdwWzXLRc0keaM6CQ,Soledad,Solo,Ricardo Arjona
539,671oWTKfzcjAwryvKwZ3QR,Si el Norte Fuera el Sur,Solo,Ricardo Arjona
540,7MCjSUK2WqLtSdsLI7AMDo,Realmente No Estoy Tan Solo,Solo,Ricardo Arjona
541,7dpCORF1xyQcSF3m5l1kJd,Señora de las Cuatro Decadas,Solo,Ricardo Arjona


In [83]:
# ID de las canciones
song_features = []
canciones_id =[canciones[i][0] for i in range(len(canciones))]

for i in canciones_id:
    features = spotify.get_audio_features(i)
    # Caracteristicas de las canciones
    #danceability=features['danceability']
    danceability = features['danceability'] if 'danceability' in features else None 
    energy =features['energy']if 'energy' in features else None 
    key=features['key']if 'key' in features else None 
    loudness=features['loudness']if 'loudness' in features else None 
    mode=features['mode']if 'mode' in features else None 
    speechiness= features['speechiness']if 'speechiness' in features else None 
    acousticness=features['acousticness']if 'acousticness' in features else None 
    instrumentalness=features['instrumentalness']if 'instrumentalness' in features else None 
    liveness=features['liveness']if 'liveness' in features else None 
    valence=features['valence']if 'valence' in features else None 
    tempo=features['tempo']if 'tempo' in features else None 
    id_ = features['id']if 'id' in features else None 

    song_features.append([danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id_])

columns =['danceability','energy','key','loudness','mode','speechiness','acousticness','instrumentalness','liveness','valence','tempo','id']
df_features = pd.DataFrame(song_features,columns = columns)
df_features

# aqui aplicar secuencia if para evitar ID Nulos.

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id
0,0.589,0.6510,6.0,-5.097,1.0,0.0306,0.02910,0.000003,0.111,0.166,90.011,7gHs73wELdeycvS48JfIos
1,0.612,0.9630,5.0,-3.912,0.0,0.0531,0.00897,0.003780,0.294,0.179,138.009,5khPVYWIdAE0kSTqekYIwB
2,0.345,0.0581,3.0,-17.217,1.0,0.0374,0.97600,0.000366,0.103,0.304,174.117,3JOVTQ5h8HGFnDdp4VT3MP
3,0.345,0.0581,3.0,-17.217,1.0,0.0374,0.97600,0.000366,0.103,0.304,174.117,3JOVTQ5h8HGFnDdp4VT3MP
4,0.454,0.9330,2.0,-6.539,1.0,0.0912,0.02400,0.042100,0.493,0.141,122.913,1Ob57NqUydp5cC3FCNV4Gy
...,...,...,...,...,...,...,...,...,...,...,...,...
538,0.419,0.3400,9.0,-7.775,0.0,0.0614,0.86600,0.000000,0.128,0.170,91.929,4UWq0SdwWzXLRc0keaM6CQ
539,0.461,0.2580,9.0,-9.911,1.0,0.0438,0.86200,0.000000,0.123,0.428,145.368,671oWTKfzcjAwryvKwZ3QR
540,0.527,0.3750,7.0,-8.100,1.0,0.0401,0.83200,0.000000,0.101,0.429,125.267,7MCjSUK2WqLtSdsLI7AMDo
541,0.533,0.2760,0.0,-9.341,1.0,0.0411,0.89600,0.000004,0.133,0.399,125.845,7dpCORF1xyQcSF3m5l1kJd


In [78]:
df_features[df_features['danceability'].isnull()]

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id
226,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
241,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None


In [82]:
df[225:242]

,id,nombre,album,artista
225,30XrHoab9KfxR0S0Ljioaf,Satania,Finisterra,Mägo de Oz
226,None,Exo-Polotics,Black Holes and Revelations,Muse
227,4uBzDO9wmPEXBKhfwAlMFJ,No Surprises,OK Computer (Collector's Edition),Radiohead
228,4SwVcoBjjawbnexIiUDLc5,Lost+ (with Jay-Z),Lost!,Coldplay
229,1dECoBMh4flNRPOvvu6Dam,Hot,Hot,INNA
230,4hszow3YBRAbGGA0WbbR6r,"Gonna Fly Now (Theme From ""Rocky"") - 2006 - Re...",Rocky (30th Anniversary Edition),Bill Conti
231,3LI4MmibTkXH5cGpCGZgyw,Teenage Dirtbag,"Songs From Dawson's Creek, Vol. II",Wheatus
232,5PO4WVTCfRUMskcwg7mamK,Immigrant Song,The Girl With the Dragon Tattoo,Trent Reznor & Atticus Ross & Karen O
233,5sYHp3PhJuFizyI4WGOwT3,Live Forever,Familiar To Millions,Oasis
234,6vrUTGn5p8IrfTZ0J6sIVM,Iris,Greatest Hits Volume One - The Singles,The Goo Goo Dolls


In [64]:
# Obtener las caracteristicas de audio
audio = spotify.get_audio_features('2U10qQg2DY80o590FcA4gx')
audio

#df_features = pd.DataFrame(audio,columns=columns,index=['id'])
#audio['danceability']

{'danceability': 0.526,
 'energy': 0.881,
 'key': 10,
 'loudness': -4.92,
 'mode': 1,
 'speechiness': 0.0454,
 'acousticness': 0.0535,
 'instrumentalness': 0.00667,
 'liveness': 0.972,
 'valence': 0.31,
 'tempo': 110.402,
 'type': 'audio_features',
 'id': '2U10qQg2DY80o590FcA4gx',
 'uri': 'spotify:track:2U10qQg2DY80o590FcA4gx',
 'track_href': 'https://api.spotify.com/v1/tracks/2U10qQg2DY80o590FcA4gx',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/2U10qQg2DY80o590FcA4gx',
 'duration_ms': 273800,
 'time_signature': 4}

In [68]:
for i in canciones:
    df_audiofeatures = pd.DataFrame(spotify.get_audio_features(i[0]),index=[range(1000)])
    
df_audiofeatures

# No funcionó


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, ...)",0.62,0.497,0,-7.443,1,0.027,0.558,0.000007,0.251,0.427,140.207,audio_features,2rgH6C08H3J2AWlckoG9Vm,spotify:track:2rgH6C08H3J2AWlckoG9Vm,https://api.spotify.com/v1/tracks/2rgH6C08H3J2...,https://api.spotify.com/v1/audio-analysis/2rgH...,218760,4


In [13]:
# Documentacion para las Playlist: 
# https://developer.spotify.com/documentation/web-api/reference/playlists/get-a-list-of-current-users-playlists/

# Maximo se pueden obtener 50 listas de reproduccion
# https://api.spotify.com/v1/me/playlists" -H "Authorization: Bearer {your access token} --> Ejemplo para extraer las listas
# Tres Llamadas Get para descargar la data 

In [14]:
# Documentacion Obtencion de info de las listas de reproduccion:
# https://developer.spotify.com/documentation/web-api/reference/tracks/get-several-audio-features/

## 3. Analisis de las listas de reproduccion

## 4. Generacion de las nuevas listas de reproduccion con canciones que tengan caracteristicas similares a las de las listas de reproduccion generadas.

In [15]:
# un metodo post para subir la lista de reproduccion

## 5. BONUS: Scrapear lista de reproduccion de wikipedia de Luke Cage y generar la lista de reproduccion pertinente